# Bubble Oscillations -- Brightfield
### Brice Saint-Michel, TU Delft
(can be reached going for bsaintmichel on gmail)

* This program goes and fetches videos to process bubble oscillation data
* Some formatting is needed to get the correct metadata for the experiments 
(i.e. include frequency, number of cycles, voltage and gain)
* Other than that most of the processing is automatic
* As the PFV software has changed during experiments I will try to keep compability with 
both versions of the .cih (or .cihx) files

In [114]:
main_folder = 'G:\\Data\\Bubble Oscillation\\20210125\\'
exp_folder  = 'Bubble'
filetypes = ('.avi', '.mp4')
show_pics = False # Displays live data processing (for monitoring)
reprocess = False # Does nothing if a saved data file is already present

### Here, we define the functions that will look into the metadata (string file + .cih file)
### of the experiment

In [133]:
import glob, re
vidfiles = (glob.glob(main_folder + '/**/*.avi', recursive=True) + 
            glob.glob(main_folder + '/**/*.mp4', recursive=True))

def parse_name(filepath):
    splits = filepath.rsplit('\\', 1)
    filename = splits[1]
    info = {'folder':splits[0], 'name':splits[1], 'ncyc':100, 'volt':00,
            'gain':0, 'numexp':0}
        # I am using default values here, they will be overwritten
        # by the program

    voltmatch = re.findall('(\d*)mV', filename)
    gainmatch = re.findall('(\d*)pc', filename)
    cycmatch = re.findall('(\d*)cyc', filename)
    numexpmatch = re.findall('_(\d*)[\W]', filename)
    
    if voltmatch:
        info['volt'] = int(voltmatch[0])
    if gainmatch: 
        info['gain'] = int(gainmatch[0])
    if cycmatch:
        info['cyc'] = int(cycmatch[0])
    if numexpmatch:
        info['numexp'] = int(numexpmatch[0])

    # Handling the cih(X) file
    cihmatch = glob.glob(info['folder'] + '\\' + info['name'].split('.')[0] + '.cih*')
    if not cihmatch:
        info['skip'] = True # There should always be a CIH(X) accompanying file with the video file
    if cihmatch[0].endswith('.cih'):
        info = readcih(cihmatch[0], info)
    elif cihmatch[0].endswith('.cihx'):
        info = readcihX(cihmatch[0], info)

    return info

# Reads Metadata file from (old) PFV version
# Data usually does NOT contain any info on **magnification** (and hence pixel size) 
def readcih(cihfilename, info):
    cihfile = open(cihfilename)
    cihdata = cihfile.read()
    cihfile.close()

    datematch = re.findall('\nDate : (\d+/\d+/\d+)', cihdata)
    timematch = re.findall('\nTime : (\d+:\d+)', cihdata)
    fpsmatch  = re.findall('\nRecord Rate\(fps\) : (\d+)', cihdata)
    shuttermatch = re.findall('Shutter Spped\(s\) : 1/(\d+)', cihdata)
    frame0match = re.findall('Start Frame : (\d+)', cihdata)
    nframesmatch = re.findall('\nTotal Frame : (\d+)', cihdata)
    decimmatch = re.findall('Save Step : (\d+)', cihdata)
    vidheightmatch = re.findall('Image Height : (\d+)', cihdata)
    vidwidthmatch = re.findall('Image Width : (\d+)', cihdata)
    
    if datematch: info['date'] = datematch[0]
    if timematch: info['time'] = timematch[0]
    if fpsmatch: info['fps'] = int(fpsmatch[0])
    if shuttermatch: info['shutter'] = int(shuttermatch[0])
    if frame0match: info['frame0'] = int(frame0match[0])
    if nframesmatch: info['nframes'] = int(nframesmatch[0])
    if decimmatch: info['decim'] = int(decimmatch[0])
    if vidwidthmatch: info['width'] = int(vidwidthmatch[0])
    if vidheightmatch: info['height'] = int(vidheightmatch[0])
    info['fps'] = info['fps']/info['decim'] # Account for decimation
    
    return info

# Reads Metadata file from (old) PFV version
# In that case I have usually specified metadata 
def readcihX(cihfilename, info):
    cihfile = open(cihfilename)
    cihdata = cihfile.read()
    cihfile.close()

    datematch = re.findall('<date>(\d+)</date>', cihdata)
    timematch = re.findall('<time>(\d+)</time>', cihdata)
    fpsmatch  = re.findall('<recordRate>(\d+)</recordRate>', cihdata)
    shuttermatch = re.findall('<shutterSpeed>(\d+)</shutterSpeed>', cihdata)
    frame0match = re.findall('<startFrame>(\d+)</startFrame>', cihdata)
    nframesmatch = re.findall('<totalFrame>(\d+)</totalFrame>', cihdata)
    decimmatch = re.findall('<skipFrame>(\d+)</skipFrame>', cihdata)
    vidheightmatch = re.findall('<height>(\d+)</height>', cihdata)
    vidwidthmatch = re.findall('<width>(\d+)</width>', cihdata)
    magnificationmatch = re.findall('<magnification>(\d+)</magnification>', cihdata)

    if datematch: info['date'] = datematch[0]
    if timematch: info['time'] = timematch[0]
    if fpsmatch: info['fps'] = int(fpsmatch[0])
    if shuttermatch: info['shutter'] = int(shuttermatch[0])
    if frame0match: info['frame0'] = int(frame0match[0])
    if nframesmatch: info['nframes'] = int(nframesmatch[0])
    if decimmatch: info['decim'] = int(decimmatch[0])
    if vidwidthmatch: info['width'] = int(vidwidthmatch[0])
    if vidheightmatch: info['height'] = int(vidheightmatch[0])
    if magnificationmatch: info['pixsize'] = 2e-5/float(magnificationmatch[0])
    info['fps'] = info['fps']/info['decim'] # Account for decimation

    return info


info = parse_name(vidfiles[0])
